In [1]:
import numpy as np

In [2]:
# normal layers
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Merge, Lambda, Activation
from keras.optimizers import RMSprop
# custom stuff
from keras import backend as K
from keras.engine.topology import Layer

Using TensorFlow backend.


In [5]:
class Bilinear(Layer):
    """ Computes $x^T zI y rowwise for a batch"""
    def __init__(self, **kwargs):
        super(Bilinear, self).__init__(**kwargs)
        
    def build(self, input_shape):
        assert len(input_shape) == 3, "Input should be shape (batch_size, 3, embed_size)"
        embed_dim = input_shape[2] # 0 is the batch dim
        self.trainable_weights = []
        
    def call(self, tensor, mask=None):
        x = tensor[:,0,:]
        y = tensor[:,1,:]
        z = tensor[:,2,:]
        xTz = Merge(mode='mul')([x, z])
        xTzy = K.batch_dot(xTz, y, axes=1)
        return xTzy
    
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], 1)

In [9]:
vocab_size = 100
target_embed_size = 100
word_embed_size = 300
dependency_embed_size = 25
pos_embed_size = 25

x_input = Input(shape=(1,), dtype='int32', name='x_input')
y_input = Input(shape=(1,), dtype='int32', name='y_input')
bilinear_input = Input(shape=(target_embed_size,), dtype='float', name='bilinear_input')

x_embed = Embedding(input_dim=vocab_size, output_dim=target_embed_size)(x_input)
y_embed = Embedding(input_dim=vocab_size, output_dim=target_embed_size)(y_input)

bilinear_expand = K.expand_dims(bilinear_input, dim=1)

xyz = Merge(mode='concat', concat_axis=1)([x_embed, y_embed, bilinear_expand])
score = Bilinear()(xyz)
output = Activation('sigmoid')(score)
# output = Lambda(lambda x:x, lambda in_shape:in_shape)(score)

bi = Model(input=[x_input, y_input, bilinear_input], output=[output])
bi.compile(optimizer=RMSprop(), loss='accuracy')

Exception: Output tensors to a Model must be Keras tensors. Found: Tensor("Sigmoid_1:0", shape=(?, ?), dtype=float32)